In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from dash.dash_table import FormatTemplate
from dash.dash_table.Format import Format, Scheme

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the database CRUD module
from PyMongo_Driver import AnimalShelter

###########################
# Rescue Filter Queries
###########################
WATER = {
    "animal_type": "Dog",
    "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
    "sex_upon_outcome": "Intact Female",
    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
}

MOUNTAIN = {
    "animal_type": "Dog",
    "breed": {"$in": ["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
    "sex_upon_outcome": "Intact Male",
    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
}

DISASTER = {
    "animal_type": "Dog",
    "breed": {"$in": ["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
    "sex_upon_outcome": "Intact Male",
    "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
}

def query_for(key: str):
    return{"water": WATER, "mountain": MOUNTAIN, "disaster": DISASTER}.get(key,{})

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

for col in ["location_lat", "location_long"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Column Headers
column_headers = {
    "animal_id": "Animal ID",
    "rec_num": "Record #",
    "animal_type": "Type",
    "breed": "Breed",
    "sex_upon_outcome": "Sex",
    "name": "Name",
    "color": "Color",
    "date_of_birth": "Birth Date",
    "age_upon_outcome": "Age (Years)",
    "age_upon_outcome_in_weeks": "Age (Weeks)",
    "outcome_subtype": "Outcome Subtype",
    "outcome_type": "Outcome Type",
    "location_lat": "Latitude",
    "location_long": "Longitude",
    "datetime": "Date/Time",
    "monthyear": "Month/Year"
}

#Formatting the columns
columns = []
for col in df.columns:
    if col not in df.columns:
        continue
    col_def = {"id": col, "name": column_headers.get(col, col), "deletable": False, "selectable": True}
    if col in ("location_lat", "location_long", "age_upon_outcome_in_weeks"):
        col_def["type"] = "numeric"
        col_def["format"] = Format(precision=2, scheme=Scheme.fixed)
    columns.append(col_def)

# Encode logo image for display
image_filename = '/home/carllalonde_snhu/Desktop/Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

header = html.Div(
    [
        html.Center(html.B(html.H1('CS-340 Dashboard'))),
        html.Center(html.A(html.Img(src=f"data:image/png;base64, {encoded_image}", style={"height": "60px"}), href="https://www.snhu.edu"),),
        html.Center(html.B(html.H1("Carl LaLonde: Project_Two_Dashboard"))),
        html.Hr(),
    ],
    style={
        "position": "sticky",
        "top": 0,
        "left": 0,
        "right": 0,
        "zIndex": 1000,
        "background": "white",
        "padding": "10px 0",
        "baxShadow": "0 2px 6px rgba(0,0,0,0.08)",
    },
)


# Web app layout
app.layout = html.Div([
    header,
    html.Div(
        
# Rescue type filter options
        html.Div([
            dcc.RadioItems(
            id="filter-type",
            options=[
                {"label": "Water Rescue", "value": "water"},
                {"label": "Mountain/Wilderness Rescue", "value": "mountain"},
                {"label": "Disaster/Individual Tracking", "value": "disaster"},
                {"label": "Reset", "value": "reset"},
            ],
            value="reset",
            inline=True
            )
        ]),

    ),
    html.Hr(),
    
    # DataTable view
    dash_table.DataTable(
        id='datatable-id',
        columns=columns,
        data=df.to_dict('records'),
        
        #Interaction for the data table    
        sort_action="native",
        filter_action="native",
        page_action="native",
        page_current=0,
        page_size=10,
        row_selectable="single",
        selected_rows=[0],
        
        #Style for the data table
        style_as_list_view=True,
        style_table={"height": "450px", "overflowY": "auto"},
        fixed_rows={'headers': True},
        style_cell={"minWidth": "120px", "width": "120px", "maxWidth": "280px",
                   "whiteSpace": "normal", "textAlign": "left", "padding": "6px"},
        style_header={"fontWeight": "bold"},
        style_cell_conditional=[
            {"if": {"column_id": "location_lat"}, "textAlign": "right"},
            {"if": {"column_id": "location_long"}, "textAlign": "right"},
            {"if": {"column_id": "age_upon_outcome_in_weeks"}, "textAlign": "right"}

        ]
    ),
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

#Updates teh data table based on upon the rescue type filter option
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])

def update_dashboard(filter_type):
    query = query_for(filter_type)
    dff = pd.DataFrame.from_records(db.read(query))
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True, errors='ignore')
    return dff.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    if not viewData:
        return html.Div("No data to display.")
    dff = pd.DataFrame(viewData)

    counts = (
        dff.groupby("breed")
          .size()
          .reset_index(name="count")
          .sort_values("count", ascending=False)
          .head(20)
    )

    fig = px.bar(counts, x="breed", y="count", title="Preferred Animals by Breed")
    fig.update_layout(xaxis_title="", yaxis_title="Count")

    return dcc.Graph(figure=fig)   
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', 'selected_columns'))

def update_styles(selected_columns):
    selected_columns = selected_columns or []
    return [{
        'if': {'column_id': col},
        'background_color': '#D2F3FF'
    } for col in selected_columns]
        
    


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):
    if not viewData:
        return html.Div("No map data.")

    dff = pd.DataFrame(viewData)

    row = index[0] if (index and len(index)) else 0
    row = max(0, min(row, len(dff) - 1))

    lat_val = dff.loc[row, "location_lat"] if "location_lat" in dff.columns else dff.iloc[row, 13]
    lon_val = dff.loc[row, "location_long"] if "location_long" in dff.columns else dff.iloc[row, 14]
    lat = pd.to_numeric(lat_val, errors="coerce")
    lon = pd.to_numeric(lon_val, errors="coerce")

    # Default center on Austin, TX but auto-center on selected animal
    center = [30.75, -97.48]
    if pd.notna(lat) and pd.notna(lon):
        center = [float(lat), float(lon)]

    breed = (dff.loc[row, "breed"] if "breed" in dff.columns else dff.iloc[row, 4])
    name  = (dff.loc[row, "name"]  if "name"  in dff.columns else dff.iloc[row, 9])

    # Returns rendered map
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=center, zoom=12,
            children=[
                dl.TileLayer(id="base-layer-id"),
                *( [dl.Marker(
                        position=center,
                        children=[
                            dl.Tooltip(str(breed) if pd.notna(breed) else ""),
                            dl.Popup([html.H1("Animal Name"),
                                      html.P(str(name) if pd.notna(name) else "Unknown")])
                        ]
                    )] if pd.notna(lat) and pd.notna(lon) else [] )
            ]
        )
    ]

app.run_server(debug=True)


In [ ]:
PORT = 8050
app.run_server(
    mode="external",
    host="127.0.0.1",
    port=PORT,
    debug=False
)